# Exercise 3: Spark SQL

## Learning Objectives
- Load data from CSV, JSON, and Parquet formats
- Execute SQL queries on DataFrames
- Join datasets from different sources
- Write results back to HDFS in optimized formats

---

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, year, month, sum, avg, count, desc
from pyspark.sql.types import *

spark = SparkSession.builder \
    .appName("Spark SQL Lab") \
    .master("yarn") \
    .config("spark.executor.memory", "1g") \
    .config("spark.executor.instances", "2") \
    .config("spark.sql.adaptive.enabled", "true") \
    .getOrCreate()

print(f"Spark SQL session ready: {spark.sparkContext.applicationId}")

Spark SQL session ready: application_1768092706713_0008


In [2]:
print(f"maxPartitionBytes  = {int(spark.conf.get('spark.sql.files.maxPartitionBytes')[:-1])/(1024*1024)} MB")
print("defaultParallelism =", spark.sparkContext.defaultParallelism)

maxPartitionBytes  = 128.0 MB
defaultParallelism = 2


## Part 1: Loading Data from Multiple Formats

In [3]:
# Load transactions from CSV
transactions = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv("hdfs:///user/student/data/transactions.csv")

print(f"Transactions: {transactions.count()} rows")
transactions.printSchema()

Transactions: 100000 rows
root
 |-- transaction_id: string (nullable = true)
 |-- transaction_date: date (nullable = true)
 |-- transaction_time: timestamp (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- unit_price: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- payment_method: string (nullable = true)
 |-- store_region: string (nullable = true)
 |-- is_online: boolean (nullable = true)



In [4]:
print("Partitions:", transactions.rdd.getNumPartitions())
print(transactions.inputFiles())

Partitions: 2
['hdfs://namenode:9000/user/student/data/transactions.csv']


In [5]:
# Load products from JSON
products = spark.read.json("hdfs:///user/student/data/catalog.json")

print(f"Products: {products.count()} rows")
products.printSchema()

Products: 500 rows
root
 |-- category: string (nullable = true)
 |-- cost_price: double (nullable = true)
 |-- in_stock: boolean (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_name: string (nullable = true)
 |-- supplier_id: string (nullable = true)
 |-- unit_price: double (nullable = true)
 |-- weight_kg: double (nullable = true)



In [6]:
# Register as temporary views for SQL
transactions.createOrReplaceTempView("transactions")
products.createOrReplaceTempView("products")

## Part 2: Running SQL Queries

In [7]:
# Basic aggregation query
spark.sql("""
    SELECT 
        store_region,
        COUNT(*) as num_transactions,
        ROUND(SUM(total_amount), 2) as total_sales,
        ROUND(AVG(total_amount), 2) as avg_sale
    FROM transactions
    GROUP BY store_region
    ORDER BY total_sales DESC
""").show()

+------------+----------------+-------------+--------+
|store_region|num_transactions|  total_sales|avg_sale|
+------------+----------------+-------------+--------+
|        West|           21575|2.831221976E7| 1312.27|
|       North|           20048|2.652123459E7| 1322.89|
|       South|           20191|2.649756461E7| 1312.35|
|     Central|           19364|2.520488752E7| 1301.64|
|        East|           18822|2.438263004E7| 1295.43|
+------------+----------------+-------------+--------+



In [8]:
# Time-based analysis
spark.sql("""
    SELECT 
        YEAR(TO_DATE(transaction_date, 'yyyy-MM-dd')) as year,
        MONTH(TO_DATE(transaction_date, 'yyyy-MM-dd')) as month,
        COUNT(*) as transactions,
        ROUND(SUM(total_amount), 2) as revenue
    FROM transactions
    GROUP BY year, month
    ORDER BY year, month
""").show(24)

+----+-----+------------+----------+
|year|month|transactions|   revenue|
+----+-----+------------+----------+
|2023|    1|        4259|5565325.87|
|2023|    2|        3776|4997062.64|
|2023|    3|        4189| 5416674.4|
|2023|    4|        3989|5202017.56|
|2023|    5|        4189|5449502.91|
|2023|    6|        4060|5405176.48|
|2023|    7|        4224|5491132.37|
|2023|    8|        4295|5545680.62|
|2023|    9|        4113|5480710.13|
|2023|   10|        4299|5666773.14|
|2023|   11|        4034|5216630.28|
|2023|   12|        4340|5689366.69|
|2024|    1|        4330|5594096.35|
|2024|    2|        3978|5238518.21|
|2024|    3|        4220|5654334.45|
|2024|    4|        4159|5423307.73|
|2024|    5|        4158| 5442786.8|
|2024|    6|        4117| 5416887.4|
|2024|    7|        4304|5548275.61|
|2024|    8|        4257|5466102.23|
|2024|    9|        4184|5493626.82|
|2024|   10|        4225|5579579.63|
|2024|   11|        4015|5349776.86|
|2024|   12|        4286|5585191.34|
+

In [9]:
# Payment method analysis
spark.sql("""
    SELECT 
        payment_method,
        is_online,
        COUNT(*) as count,
        ROUND(AVG(total_amount), 2) as avg_amount
    FROM transactions
    GROUP BY payment_method, is_online
    ORDER BY payment_method, is_online
""").show()

+--------------+---------+-----+----------+
|payment_method|is_online|count|avg_amount|
+--------------+---------+-----+----------+
|          Cash|    false|10025|   1299.04|
|          Cash|     true|10114|   1308.34|
|   Credit Card|    false|10009|   1309.59|
|   Credit Card|     true|10035|   1320.26|
|    Debit Card|    false| 9847|   1302.93|
|    Debit Card|     true| 9886|   1304.85|
|     Gift Card|    false| 9889|   1316.72|
|     Gift Card|     true|10240|   1322.57|
|        PayPal|    false|10002|    1293.2|
|        PayPal|     true| 9953|    1314.0|
+--------------+---------+-----+----------+



## Part 3: Joining Datasets

In [10]:
# # Join transactions with products
sales_with_products = spark.sql("""
    SELECT 
        t.transaction_id,
        t.transaction_date,
        p.product_name,
        p.category,
        t.quantity,
        t.unit_price,
        t.total_amount,
        p.cost_price * t.quantity as total_cost,
        t.total_amount - (p.cost_price * t.quantity) as profit
    FROM transactions t
    JOIN products p ON t.product_id = p.product_id
""")

sales_with_products.show(10)

+--------------+----------------+--------------------+---------------+--------+----------+------------+------------------+------------------+
|transaction_id|transaction_date|        product_name|       category|quantity|unit_price|total_amount|        total_cost|            profit|
+--------------+----------------+--------------------+---------------+--------+----------+------------+------------------+------------------+
|   TXN00000001|      2024-06-03|Home & Garden Ite...|  Home & Garden|       1|    259.06|      259.06|            139.12|            119.94|
|   TXN00000002|      2023-11-22|     Office Item 288|         Office|       6|     442.7|      2656.2|1114.8600000000001|1541.3399999999997|
|   TXN00000003|      2023-08-13|Health & Beauty I...|Health & Beauty|       1|     116.0|       116.0|             50.17|             65.83|
|   TXN00000004|      2023-06-20|Electronics Item 230|    Electronics|       4|    187.53|      750.12|            364.64|            385.48|
|   TX

In [30]:
# Profit by category
sales_with_products.createOrReplaceTempView("sales_detail")

spark.sql("""
    SELECT 
        category,
        COUNT(*) as transactions,
        ROUND(SUM(total_amount), 2) as revenue,
        ROUND(SUM(profit), 2) as total_profit,
        ROUND(AVG(profit), 2) as avg_profit_per_sale
    FROM sales_detail
    GROUP BY category
    ORDER BY total_profit DESC
""").show()

+---------------+------------+-------------+------------+-------------------+
|       category|transactions|      revenue|total_profit|avg_profit_per_sale|
+---------------+------------+-------------+------------+-------------------+
|         Office|       11109|1.663535544E7|   7347119.5|             661.37|
|       Clothing|       11712|1.585799722E7|  6517862.36|             556.51|
|         Sports|       11340|1.409068302E7|  6133215.62|             540.85|
|     Automotive|       10433|1.335601151E7|  6048683.21|             579.76|
|Food & Beverage|        9554|1.285170106E7|  5626330.56|              588.9|
|           Toys|        8827| 1.20920478E7|  5399772.47|             611.73|
|Health & Beauty|       10497|1.270791491E7|  5388642.39|             513.35|
|  Home & Garden|       10196|1.318413938E7|  5379350.34|             527.59|
|          Books|        9101|1.147227731E7|  4686487.61|             514.94|
|    Electronics|        7231|   8670408.87|  3607807.03|       

## Part 4: Writing Data in Optimized Formats

In [28]:
# Write to Parquet (columnar, compressed)
sales_with_products.write \
    .mode("overwrite") \
    .parquet("hdfs:///user/student/output/sales_parquet")

print("Written to Parquet!")

Written to Parquet!


In [25]:
# Write partitioned by category (useful for filtering)
sales_with_products.write \
    .mode("overwrite") \
    .partitionBy("category") \
    .parquet("hdfs:///user/student/output/sales_by_category")

print("Written with partitioning!")

Written with partitioning!


In [26]:
# Check the output
!hdfs dfs -ls /user/student/output/sales_by_category/

Found 11 items
-rw-r--r--   3 jovyan supergroup          0 2026-01-11 13:52 /user/student/output/sales_by_category/_SUCCESS
drwxr-xr-x   - jovyan supergroup          0 2026-01-11 13:52 /user/student/output/sales_by_category/category=Automotive
drwxr-xr-x   - jovyan supergroup          0 2026-01-11 13:52 /user/student/output/sales_by_category/category=Books
drwxr-xr-x   - jovyan supergroup          0 2026-01-11 13:52 /user/student/output/sales_by_category/category=Clothing
drwxr-xr-x   - jovyan supergroup          0 2026-01-11 13:52 /user/student/output/sales_by_category/category=Electronics
drwxr-xr-x   - jovyan supergroup          0 2026-01-11 13:52 /user/student/output/sales_by_category/category=Food & Beverage
drwxr-xr-x   - jovyan supergroup          0 2026-01-11 13:52 /user/student/output/sales_by_category/category=Health & Beauty
drwxr-xr-x   - jovyan supergroup          0 2026-01-11 13:52 /user/student/output/sales_by_category/category=Home & Garden
drwxr-xr-x   - jovyan supergr

In [27]:
# Read back and verify
parquet_data = spark.read.parquet("hdfs:///user/student/output/sales_parquet")
print(f"Read back {parquet_data.count()} rows from Parquet")

Read back 100000 rows from Parquet


In [ ]:
spark.stop()